In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset, drop_first=True, dtype=int)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID", axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
independent=dataset[["Age","EstimatedSalary","Gender_Male"]]
dependent=dataset["Purchased"]

In [10]:
independent.shape

(400, 3)

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent, dependent, test_size = 0.3, random_state = 0)

In [14]:
from sklearn.neighbors import KNeighborsClassifier

In [26]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors':[10],
              'p':[2],
              'metric':['minkowski']}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')

grid.fit(x_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


Exception ignored in: <function ResourceTracker.__del__ at 0x102ba9bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['minkowski'], 'n_neighbors': [10],
                         'p': [2]},
             scoring='f1_weighted', verbose=3)

In [27]:
re=grid.cv_results_

grid_predictions = grid.predict(x_test) 

In [28]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [29]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[76  3]
 [16 25]]


In [30]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, y_pred)

In [31]:
print("The Classification Report:\n", clf_report)

The Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.96      0.89        79
           1       0.89      0.61      0.72        41

    accuracy                           0.84       120
   macro avg       0.86      0.79      0.81       120
weighted avg       0.85      0.84      0.83       120



In [32]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test, grid_predictions, average='weighted')

In [33]:
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'metric': 'minkowski', 'n_neighbors': 10, 'p': 2}: 0.8327697262479871


In [34]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, grid.predict_proba(x_test)[:,1])

np.float64(0.8519604816301327)